In [120]:
import numpy as np
import matplotlib.pyplot as plt
import inspect

In [121]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../', '../', '../', '../', '../', '../', '../']


In [122]:
from IPython.display import display, HTML

display(HTML('''
<style>
.output {
    display: flex;
    flex-direction: column;
}
.output_scroll {
    max-height: 300px; 
    overflow-y: auto;
}
</style>
'''))


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [123]:
x_train, y_train,x_test_original, y_test_original = ReadTS('EOGHorizontalSignal')
y_train = y_train - 1
y_test_original = y_test_original - 1
#print(x_train.shape)
print(y_train)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  5.  5.  5.  5.
  5.  5.  5.  5.  5.  5.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  7.  7.
  7.  7.  7.  7.  7.  7.  7.  7.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.
  9.  9.  9.  9.  9.  9.  9.  9.  9.  9. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  2.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  4.  4.
  4.  4.  4.  4.  4.  4.  4.  4.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  7.  7.  7.  7.  7.  7.  7.  7.
  7.  7.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  9.  9.  9.  9.  9.  9.
  9.  9.  9.  9. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 11. 11. 11. 11.
 11. 11. 11. 11. 11. 11.  0.  0.  0.  0.  0.  0.  0

In [124]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):

    original_shape = TS.shape
    

    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    

    scaler = StandardScaler()
    

    TS_normalized = scaler.fit_transform(TS_2d)

    
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)

In [135]:
model = make_CNN_model(x_train.shape[1:],num_classes = 12)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=300, batch_size=32)

Epoch 1/300
10/10 [==============================] - 1s 46ms/step - loss: 2.3436 - sparse_categorical_accuracy: 0.1349 - val_loss: 2.4481 - val_sparse_categorical_accuracy: 0.0822
Epoch 2/300
10/10 [==============================] - 0s 40ms/step - loss: 2.0243 - sparse_categorical_accuracy: 0.2976 - val_loss: 2.4325 - val_sparse_categorical_accuracy: 0.0411
Epoch 3/300
10/10 [==============================] - 0s 39ms/step - loss: 1.9422 - sparse_categorical_accuracy: 0.3149 - val_loss: 2.4282 - val_sparse_categorical_accuracy: 0.0137
Epoch 4/300
10/10 [==============================] - 0s 40ms/step - loss: 1.8961 - sparse_categorical_accuracy: 0.3218 - val_loss: 2.4276 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/300
10/10 [==============================] - 0s 40ms/step - loss: 1.8621 - sparse_categorical_accuracy: 0.2976 - val_loss: 2.4266 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 6/300
10/10 [==============================] - 0s 39ms/step - loss: 1.8283 - sparse_ca

In [136]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

12/12 [==============================] - 0s 9ms/step - loss: 2.7440 - sparse_categorical_accuracy: 0.2072
Test accuracy 0.20718231797218323
Test loss 2.7440216541290283


### Create perturbed test sets and calculate decrease in accuracy

In [127]:
print(y_test_original.shape)

(362,)


In [128]:
print(inspect.getsource(perturb))
print(inspect.signature(perturb))

def perturb(perturbation_strategy, ts, index0, index1, global_ts = [],bg = []):
    if perturbation_strategy == 'RBP':
        return RBPIndividual(ts, index0, index1)
    if perturbation_strategy == 'zero':
        return zeroPerturb(ts, index0, index1)
    if perturbation_strategy == 'noise':
        return noisePerturb(ts, index0, index1)
    if perturbation_strategy == 'blur':
        return blurPerturb(ts, index0, index1)
    if perturbation_strategy == 'RBP1':
        return RBPIndividualNew1(global_ts, ts, index0, index1) 
    if perturbation_strategy == 'RBP1fast':
        return RBPIndividualNew1fast(bg, ts, index0, index1)  
    if perturbation_strategy == 'RBP2':
        return RBPIndividualNew2(global_ts, ts, index0, index1)    

(perturbation_strategy, ts, index0, index1, global_ts=[], bg=[])


In [129]:
predictions_original = model.predict(x_test_original)
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 0
index_1 = 90

x_test_rbp = np.asarray([perturb('RBP',x,index_0,index_1) for x in perturb_test_set])
x_test_rbp = np.expand_dims(x_test_rbp, axis=-1)
predictions_rbp = model.predict(x_test_rbp)

pert1 = perturb('RBP1',perturb_test_set[0],index_0,index_1, global_ts = perturb_test_set)
x_test_new1 = np.asarray([perturb('RBP1',x,index_0,index_1, global_ts = perturb_test_set) for x in perturb_test_set])
x_test_new1 = np.expand_dims(x_test_new1, axis=-1)
predictions_new1 = model.predict(x_test_new1)

#x_test_new2 = np.asarray([perturb('RBP2',x,400,500, global_ts = perturb_test_set) for x in perturb_test_set])
#x_test_new2 = np.expand_dims(x_test_new2, axis=-1)
#predictions_new2 = model.predict(x_test_new2)

x_test_zero = np.asarray([perturb('zero',x,index_0,index_1) for x in perturb_test_set])
x_test_zero = np.expand_dims(x_test_zero, axis=-1)
predictions_zero = model.predict(x_test_zero)

x_test_noise = np.asarray([perturb('noise',x,index_0,index_1) for x in perturb_test_set])
x_test_noise = np.expand_dims(x_test_noise, axis=-1)
predictions_noise = model.predict(x_test_noise)

x_test_blur = np.asarray([perturb('blur',x,index_0,index_1) for x in perturb_test_set])
x_test_blur = np.expand_dims(x_test_blur, axis=-1)
predictions_blur = model.predict(x_test_blur)

12/12 [==============================] - 0s 8ms/step


In [130]:
def l2norm(perturb):
    # 计算两个数组之间的差
    difference = perturb - x_test_original
    
    # 计算差的平方
    squared_difference = np.square(difference)
    
    # 计算平方和，沿着特征维度（axis=1），因为每个时间序列样本是一行
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    # 计算平方和的平方根，即l2范数
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    # l2_norms现在是一个形状为(467,)的数组，其中每个元素代表
    # 对应时间序列样本之间的l2范数
    print(average_l2_norm)  # 输出: (467,)

l2norm(x_test_rbp)
l2norm(x_test_new1)
l2norm(x_test_zero)
l2norm(x_test_noise)
l2norm(x_test_blur)

3.8374587446759114
6.5504900045412455
6.535232206585224
12.997963624657048
0.3219547708548638


In [131]:
def accuracy(predictions, labels):
    hits = 0
    for i in range(0,len(predictions)):
        if predictions[i] == labels[i]:
            hits += 1
    return hits/len(predictions)

In [132]:
original_accuracy = accuracy(np.argmax(predictions_original,axis=1), y_test_original)
rbp_accuracy = accuracy(np.argmax(predictions_rbp,axis=1), y_test_original)
new1_accuracy = accuracy(np.argmax(predictions_new1,axis=1), y_test_original)
#new2_accuracy = accuracy(np.argmax(predictions_new2,axis=1), y_test_original)
zero_accuracy = accuracy(np.argmax(predictions_zero,axis=1), y_test_original)
noise_accuracy = accuracy(np.argmax(predictions_noise,axis=1), y_test_original)
blur_accuracy = accuracy(np.argmax(predictions_blur,axis=1), y_test_original)

In [133]:
print(" Original Accuracy: " + str(original_accuracy) + "\n"+
    " RBP Accuracy: " + str(rbp_accuracy) + "\n"+
    " RBP1 Accuracy: " + str(new1_accuracy) + "\n"+
    #" RBP2 Accuracy: " + str(new2_accuracy) + "\n"+
      
    " Zero Accuracy: " + str(zero_accuracy) + "\n"+
    " Noise Accuracy: " + str(noise_accuracy) + "\n"+
    " Blur Accuracy: " + str(blur_accuracy) + "\n")

 Original Accuracy: 0.19889502762430938
 RBP Accuracy: 0.22099447513812154
 RBP1 Accuracy: 0.22099447513812154
 Zero Accuracy: 0.21823204419889503
 Noise Accuracy: 0.212707182320442
 Blur Accuracy: 0.20441988950276244



In [134]:
print(np.argmax(predictions_noise,axis=1))
print(np.argmax(predictions_new1,axis=1))

[4 1 2 1 4 4 0 4 0 2 2 4 4 4 2 4 6 2 4 1 2 2 2 2 2 2 2 1 6 0 1 2 1 1 6 4 4
 0 1 4 4 4 1 3 1 1 4 4 0 2 1 1 1 4 1 4 1 1 1 1 1 2 6 2 2 6 6 6 2 2 4 4 0 0
 1 0 0 0 0 0 4 4 1 1 3 1 3 3 7 1 1 4 4 3 7 4 2 1 1 0 4 2 1 1 2 2 2 2 2 2 2
 2 1 1 1 1 1 7 1 1 1 0 4 0 4 7 4 0 4 0 4 1 3 1 1 1 1 1 1 4 4 2 2 2 1 2 2 2
 2 2 1 0 0 0 0 4 7 4 3 4 4 4 7 1 3 4 4 1 4 7 4 4 1 1 1 1 1 4 1 1 1 2 2 2 2
 2 2 6 2 2 2 0 7 0 7 4 4 0 0 1 4 0 7 4 7 0 4 7 0 4 4 4 7 4 0 0 4 0 4 4 4 2
 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 4 4 4 4 4 4 0 0 4 1 4 1 1 1 1 1 1 1 1
 1 1 2 2 2 2 2 2 2 2 2 2 0 1 7 0 0 0 4 4 0 4 1 1 0 3 4 3 7 4 4 4 0 1 1 1 1
 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 7 0 4 4 0 7 0 7 7 7 7 7 3 7 4 4 3 0 4 3 0
 0 0 4 4 0 4 0 0 4 6 2 2 2 2 6 6 2 2 2 1 1 1 1 3 1 1 1 1 1]
[ 6  1  2  1  4  7  0  3  0  2  2  1  3  7  2  3  6  6  3  1  2  2  6  2
  2  6  6  1  6  0  1  2  1  1  6  1  5  8  1  4  1  0  1  3  1  1  0  3
  0  2  1  1  1  1  1  7  1  1  1  1  1  6  6  2  6  6  6  6  2  2  3  4
  0  0  1  0  0  0  0  0  3  7 11 11  3 11  3 